In [16]:
import os
import glob
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# 花朵圖片下載網址
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

# 用 keras API 下載花朵圖片
zip_file = tf.keras.utils.get_file(origin=_URL,                 # 指定下載網址
                                   fname="flower_photos.tgz",   # 命名檔案
                                   extract=True)                # 是否解壓縮

# 取得圖片資料夾路徑
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

# 該資料共有 5 個類別
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

228818944/228813984 [==============================] - 32s 0us/step


In [4]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)                          # 取得單一類別資料夾路徑
  images = glob.glob(img_path + '/*.jpg')                        # 載入所有 jpg 檔成為一個 list
  print("{}: {} Images".format(cl, len(images)))                 # 印出單一類別有幾張圖片
  num_train = int(round(len(images)*0.8))                        # 切割 80% 資料作為訓練集
  train, val = images[:num_train], images[num_train:]            # 訓練 > 0~80%，驗證 > 80%~100%

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):  # 如果資料夾不存在
      os.makedirs(os.path.join(base_dir, 'train', cl))           # 建立新資料夾
    shutil.move(t, os.path.join(base_dir, 'train', cl))          # 搬運圖片資料到新的資料夾

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):    # 如果資料夾不存在
      os.makedirs(os.path.join(base_dir, 'val', cl))             # 建立新資料夾
    shutil.move(v, os.path.join(base_dir, 'val', cl))            # 搬運圖片資料到新的資料夾

roses: 641 Images
daisy: 633 Images
dandelion: 898 Images
sunflowers: 699 Images
tulips: 799 Images


In [5]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [6]:
batch_size = 100
IMG_SHAPE = 150 

In [10]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,               # 從0~255整數，壓縮為0~1浮點數
    rotation_range=45,            # 隨機旋轉 ±45°
    width_shift_range=.15,        # 隨機水平移動 ±15%
    height_shift_range=.15,       # 隨機垂直移動 ±15%
    horizontal_flip=True,         # 隨機水平翻轉
    zoom_range=0.5                # 隨機縮放 50%
)


train_data_gen = image_gen_train.flow_from_directory(
    batch_size=batch_size,             # batch_size=100
    directory=train_dir,               # 指定訓練集的資料夾路徑
    shuffle=True,                      # 洗牌，打亂圖片排序
    target_size=(IMG_SHAPE,IMG_SHAPE),  # 圖片大小統一為 150 * 150 pixel
    class_mode='sparse'           # 分類標籤定義為 0, 1, 2, 3, 4
)

Found 2935 images belonging to 5 classes.


In [11]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(
    batch_size=batch_size,
    directory=val_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='sparse'
)

Found 735 images belonging to 5 classes.


In [19]:
model = Sequential()

model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [21]:
epochs = 50

history = model.fit(
    train_data_gen,               # 帶入訓練資料產生器
    epochs=epochs,                # 將所有資料看過 50 次
    validation_data=val_data_gen  # 帶入驗證資料產生器
)

Epoch 1/50
30/30 [==============================] - 18s 607ms/step - loss: 0.9418 - acc: 0.6320 - val_loss: 0.8523 - val_acc: 0.6939
Epoch 2/50
30/30 [==============================] - 16s 541ms/step - loss: 0.9166 - acc: 0.6416 - val_loss: 0.8327 - val_acc: 0.6857
Epoch 3/50
30/30 [==============================] - 16s 542ms/step - loss: 0.8834 - acc: 0.6480 - val_loss: 0.7908 - val_acc: 0.7102
Epoch 4/50
30/30 [==============================] - 18s 587ms/step - loss: 0.8776 - acc: 0.6675 - val_loss: 0.8192 - val_acc: 0.6694
Epoch 5/50
30/30 [==============================] - 17s 564ms/step - loss: 0.8413 - acc: 0.6716 - val_loss: 0.7317 - val_acc: 0.7265
Epoch 6/50
30/30 [==============================] - 18s 590ms/step - loss: 0.8035 - acc: 0.6927 - val_loss: 0.7370 - val_acc: 0.7224
Epoch 7/50
30/30 [==============================] - 18s 615ms/step - loss: 0.7865 - acc: 0.6903 - val_loss: 0.7374 - val_acc: 0.7197
Epoch 8/50
30/30 [==============================] - 20s 652ms/step - 